In [1]:
import numpy as np
import pandas as pd
from pandas import ExcelWriter
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
df_1 = pd.read_excel("excel_1.xlsx", sheet_name=0)
df_2 =  pd.read_excel("excel_2.xlsx", sheet_name=0)

In [3]:
df_1

,Name,DOB,12/1,12/2,12/3,12/4,12/5,12/6,12/7,12/8,12/9,12/10,12/11,12/12,12/13,12/14,12/15,12/16,12/17,12/18,12/19,12/20,12/21,12/22,12/23,12/24,12/25,12/26,12/27,12/28,12/29,12/30,12/31
0,Mahesh,2021-01-10,Pr,Ab,Pr,Pr,Pr,Pr,Pr,Pr,Pr,Ab,Ab,Ab,Ab,Ab,Ab,Ab,Ab,Pr,Pr,Ab,Ab,Ab,Ab,Ab,Ab,Ab,Ab,Ab,Ab,Ab,Ab
1,Shri,2021-01-11,Ab,Ab,Pr,Pr,Pr,Pr,Pr,Pr,Pr,Ab,Pr,Ab,Ab,Ab,Ab,Ab,Ab,Pr,Pr,Ab,Ab,Ab,Ab,Ab,Ab,Ab,Ab,Ab,Ab,Ab,Ab
2,Raj,2021-01-12,Ab,Ab,Pr,Pr,Pr,Pr,Pr,Pr,Pr,Ab,Pr,Ab,Ab,Ab,Ab,Ab,Ab,Pr,Pr,Ab,Ab,Ab,Ab,Ab,Ab,Pr,Pr,Pr,Pr,Pr,Pr
3,Yash,2021-01-13,Ab,Ab,Pr,Pr,Pr,Pr,Pr,Pr,Pr,Ab,Pr,Ab,Ab,Ab,Ab,Ab,Ab,Pr,Pr,Ab,Ab,Ab,Ab,Ab,Ab,Pr,Pr,Pr,Pr,Pr,Pr
4,Nik,2021-01-14,Ab,Ab,Pr,Pr,Pr,Pr,Pr,Pr,Pr,Ab,Pr,Ab,Ab,Ab,Ab,Ab,Ab,Pr,Pr,Ab,Ab,Ab,Ab,Ab,Ab,Ab,Ab,Ab,Ab,Ab,Ab
5,Saurabh,2021-01-15,Ab,Ab,Pr,Pr,Pr,Pr,Pr,Pr,Pr,Ab,Pr,Ab,Ab,Ab,Ab,Ab,Ab,Pr,Pr,Ab,Ab,Ab,Ab,Ab,Ab,Pr,Pr,Pr,Pr,Pr,Pr


In [4]:
df_2

,Name,DOB,12/1,12/2,12/3,12/4,12/5,12/6,12/7,12/8,12/9,12/10,12/11,12/12,12/13,12/14,12/15,12/16,12/17,12/18,12/19,12/20,12/21,12/22,12/23,12/24,12/25,12/26,12/27,12/28,12/29,12/30,12/31
0,Yash,2021-01-13,Ab,Ab,Pr,Pr,Pr,Pr,Pr,Pr,Pr,Ab,Pr,Ab,Ab,Ab,Ab,Ab,Ab,Pr,Pr,Ab,Ab,Ab,Ab,Ab,Ab,Pr,Pr,Pr,Pr,Pr,Pr
1,Shree,2021-01-11,Ab,Ab,Pr,Pr,Pr,Pr,Pr,Pr,Pr,Ab,Pr,Ab,Ab,Ab,Ab,Ab,Ab,Pr,Pr,Ab,Ab,Ab,Ab,Ab,Ab,Ab,Ab,Ab,Ab,Ab,Ab
2,Raj,2021-01-12,Ab,Ab,Pr,Pr,Pr,Pr,Pr,Pr,Pr,Ab,Pr,Ab,Ab,Ab,Ab,Ab,Ab,Pr,Pr,Ab,Ab,Ab,Ab,Ab,Ab,Pr,Pr,Pr,Pr,Pr,Pr
3,Saurabh,2021-01-15,Ab,Ab,Pr,Pr,Pr,Pr,Pr,Pr,Pr,Ab,Pr,Ab,Ab,Ab,Ab,Ab,Ab,Pr,Pr,Ab,Ab,Ab,Ab,Ab,Ab,Pr,Pr,Pr,Pr,Pr,Pr
4,Nik,2021-01-14,Ab,Ab,Pr,Pr,Pr,Pr,Pr,Pr,Pr,Ab,Pr,Ab,Ab,Ab,Ab,Ab,Ab,Pr,Pr,Ab,Ab,Ab,Ab,Ab,Ab,Ab,Ab,Ab,Ab,Ab,Ab
5,Mahes,2021-01-10,Ab,Ab,Pr,Pr,Pr,Pr,Pr,Pr,Pr,Ab,Ab,Ab,Ab,Ab,Ab,Ab,Ab,Pr,Pr,Ab,Ab,Ab,Ab,Ab,Ab,Ab,Ab,Ab,Ab,Ab,Ab


In [5]:
#make the result data frame with shape is equel to df_1.shape
result_df = pd.DataFrame(data = np.NaN, index = df_1.index,columns=df_1.columns)
for i in df_1.index:
    for j in df_2.index:
        # if statement checks the key('Name'+ DOB) of df_1 row with every row in df_2 
        if (df_1.iloc[i,:2].apply(lambda x:str(x).strip()) ==df_2.iloc[j,:2].apply(lambda x:str(x).strip())).all():
            #appending the value TRUE, FALSE by checking series of i'th row of df_1 to series of j'th row of df_2
            result_df.iloc[i,:] = (df_1.iloc[i,:].apply(lambda x:str(x).strip()) == df_2.iloc[j,:].apply(lambda x:str(x).strip())).values
            break
        else:
            result_df.iloc[i,:] = "key not found"

In [6]:
result_df

,Name,DOB,12/1,12/2,12/3,12/4,12/5,12/6,12/7,12/8,12/9,12/10,12/11,12/12,12/13,12/14,12/15,12/16,12/17,12/18,12/19,12/20,12/21,12/22,12/23,12/24,12/25,12/26,12/27,12/28,12/29,12/30,12/31
0,key not found,key not found,key not found,key not found,key not found,key not found,key not found,key not found,key not found,key not found,key not found,key not found,key not found,key not found,key not found,key not found,key not found,key not found,key not found,key not found,key not found,key not found,key not found,key not found,key not found,key not found,key not found,key not found,key not found,key not found,key not found,key not found,key not found
1,key not found,key not found,key not found,key not found,key not found,key not found,key not found,key not found,key not found,key not found,key not found,key not found,key not found,key not found,key not found,key not found,key not found,key not found,key not found,key not found,key not found,key not found,key not found,key not found,key not found,key not found,key not found,key not found,key not found,key not found,key not found,key not found,key not found
2,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True
3,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True
4,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True
5,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True


In [7]:
result2_df = pd.DataFrame(data= 0, index= df_1.index, columns=['Number of time key'])
for i in df_1.index:
    for j in df_2.index:
        if (df_1.iloc[i,:2] == df_2.iloc[j,:2]).all():
            result2_df.iloc[i,0] = result2_df.iloc[i,0]+1
  


In [8]:
result2_df

,Number of time key
0,0
1,0
2,1
3,1
4,1
5,1


In [11]:
# not_found_keys are the keys from df_2(extracted) which are not found in df_1(golden data)
not_found_key = []
for i in df_2.index:
    for j in df_1.index:
        if (df_2.iloc[i,:2] == df_1.iloc[j,:2]).all():
            break
        else:
            pass
    else:
        not_found_key.append(df_2.iloc[i,:2])



In [13]:
not_found_key

[Name                  Shree
 DOB     2021-01-11 00:00:00
 Name: 1, dtype: object, Name                  Mahes
 DOB     2021-01-10 00:00:00
 Name: 5, dtype: object]